In [ ]:
# For AI Practicum. GreyJack

In [ ]:
# Code for importing Python Notebooks as modules.
# Copy this into any new Player file that you create
import io, os, sys, types
from IPython import get_ipython
from nbformat import current
from IPython.core.interactiveshell import InteractiveShell


def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path


class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = current.read(f, 'json')


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.worksheets[0].cells:
                if cell.cell_type == 'code' and cell.language == 'python':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.input)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

In [ ]:
import Player

In [ ]:
class Human(Player.iPlayer):
    def __init__(self, money, name='lowly human'):
        """An instance of a human player."""
        super().__init__(money,name)

    def initial_bet(self):
        """prompts human player for initial bet, return integer."""
        while True:
            ans = str(input('How much do you want to bet?'))
            if not ans.isdigit():
                print('That\'s not real money!')
            else:
                bet = int(ans)
                if bet>self.money:
                    print('You cannot bet over what you have!')
                else:
                    self.money-=bet
                    return bet
    
    def to_hit(self, state, hand):
        """ Returns True if the player hits, False otherwise."""
        while True:
            print('Choose your action from below:\n\n'+
                  '- "hit"   or   "fold"\n' + 
                  '- "look" to see the game state.\n')
            ans = str(input())
            if ans in ['look','"look"','l']:
                self.print_game_state()
            elif ans in ['hit','"hit"','h']:
                return True
            elif ans in ['fold','"fold"','f']:
                return False
            else:
                print("Sorry, I'm afraid I cannot do that...because I don't understand.")

    def print_game_state(self):
        """prints the state visible to this player."""
        s = self.get_state()
        
    def get_state(self):
        """returns a summary of the state visible to this player."""
        pass